In [1]:
import torch
import mmcv
from mmcv import Config
from mmcv.runner import load_checkpoint
from mmseg.apis import inference_segmentor
from mmseg.datasets import build_dataset, build_dataloader
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmseg.apis import single_gpu_test
from mmseg.datasets.builder import build_dataloader
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel

import os.path as osp
import numpy as np

import numpy as np
from PIL import Image

/home/austin/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
# Load trained model
config = "../mmsegmentation/configs/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes.py"
#config = "../experimentation/configs/3026-05-17-16-25-52_CityscapesDataset_FCNHead_ResNet.py"
#config = "../experimentation/configs/3026-05-17-16-25-52_CityscapesDataset_FCNHead_ResNet.py"
#config_root = "configs/" + config

checkpoint = "work_dirs/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth"
#checkpoint = "work_dirs/experimentation/3026-05-17-16-25-52_CityscapesDataset_FCNHead_ResNet/best_mIoU_epoch_250.pth"
#checkpoint = "work_dirs/experimentation/3026-05-17-16-25-52_CityscapesDataset_FCNHead_ResNet/best_mIoU_epoch_108.pth"
cfg = Config.fromfile(config)

data_root = "../../../../../mnt/data/cityscapes"
#data_root = "../image generation/test_images/frankfurt_real_seg/"
#data_root = "./data/Bremen/"

cfg.data.val['data_root'] = data_root
datasets = [build_dataset(cfg.data.val)]

palette = datasets[0].PALETTE
classes = datasets[0].CLASSES

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint)
checkpoint['meta']['CLASSES'] = classes
checkpoint['meta']['PALETTE'] = palette

cfg.data.test['data_root'] = data_root
datasets = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        datasets,
        samples_per_gpu=1,
        workers_per_gpu=1)
model.CLASSES = datasets.CLASSES
model = MMDataParallel(model, device_ids=[0])

#output_folder = "./results_check/configs" + config
output_folder = "./results_output/segformer"

single_gpu_test(model, data_loader, opacity= 1,out_dir=output_folder)


2023-10-20 20:06:20,652 - mmseg - INFO - Loaded 500 images


/home/austin/honours/mmsegmentation/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: work_dirs/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth


2023-10-20 20:06:21,503 - mmseg - INFO - Loaded 500 images


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 500/500, 1.5 task/s, elapsed: 334s, ETA:     0s

[array([[2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        ...,
        [0, 0, 0, ..., 2, 2, 2],
        [0, 0, 0, ..., 2, 2, 2],
        [0, 0, 0, ..., 2, 2, 2]]),
 array([[8, 8, 8, ..., 2, 2, 2],
        [8, 8, 8, ..., 2, 2, 2],
        [8, 8, 8, ..., 2, 2, 2],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[2, 2, 2, ..., 7, 7, 7],
        [2, 2, 2, ..., 7, 7, 7],
        [2, 2, 2, ..., 7, 7, 7],
        ...,
        [9, 9, 9, ..., 9, 9, 9],
        [9, 9, 9, ..., 9, 9, 9],
        [9, 9, 9, ..., 9, 9, 9]]),
 array([[8, 8, 8, ..., 2, 2, 2],
        [8, 8, 8, ..., 2, 2, 2],
        [8, 8, 8, ..., 2, 2, 2],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[8, 8, 8, ..., 8, 8, 8],
        [8, 8, 8, ..., 8, 8, 8],
        [8, 8, 8, ..., 8, 8, 8],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
   

In [5]:
# RGB to mask process based on: 
# https://github.com/nikhilroxtomar/RGB-Mask-to-Single-Channel-Mask-for-Multiclass-Segmentation/blob/main/rgb_mask_to_single_channel_mask.py
def rgb_label_to_single_channel(rgb_mask, colormap):
    """
    rgb_mask: RGB image as array
    colormap: class colour palette
    """    
    output_mask = []

    for i, color in enumerate(colormap):
        cmap = np.all(np.equal(rgb_mask, color), axis=-1)
        output_mask.append(cmap)

    output_mask = np.stack(output_mask, axis=-1)
    grayscale_mask = np.argmax(output_mask, axis=-1)
    return grayscale_mask

# Convert (overwrite) annotations to 1 channel instead of 3 chanel RGB images. 
def rgbToAnnotation(data_root, ann_dir, palette):
    print(osp.join(data_root, ann_dir))
    for file in mmcv.scandir(osp.join(data_root, ann_dir), suffix=".png"):
        img = Image.open(osp.join(data_root, ann_dir, file))
        mask = rgb_label_to_single_channel(np.asarray(img), palette).astype(np.uint8)
        seg_img = Image.fromarray(mask).convert("P")
        seg_img.putpalette(np.array(palette, dtype=np.uint8))
        seg_img.save(osp.join(data_root, ann_dir, file))

def palette_colour_to_gray_save(data_root, ann_dir, save_dir):
    for file in mmcv.scandir(osp.join(data_root, ann_dir), suffix=".png"):
            img = Image.open(osp.join(data_root, ann_dir, file))
            arr = np.asarray(img)
            img_new = Image.fromarray(arr.astype(np.uint8))
            img_new.save(save_dir + str(file))

In [6]:
#id
palette = [
    (0, 0, 0),
    (0, 0, 0),
    (0, 0, 0),
    (0, 0, 0),
    (0, 0, 0),
    (111, 74, 0),
    (81, 0, 81),
    (128, 64, 128),
    (244, 35, 232),
    (250, 170, 160),
    (230, 150, 140),
    (70, 70, 70),
    (102, 102, 156),
    (190, 153, 153),
    (180, 165, 180),
    (150, 100, 100),
    (150, 120, 90),
    (153, 153, 153),
    (153, 153, 153),
    (250, 170, 30),
    (220, 220, 0),
    (107, 142, 35),
    (152, 251, 152),
    (70, 130, 180),
    (220, 20, 60),
    (255, 0, 0),
    (0, 0, 142),
    (0, 0, 70),
    (0, 60, 100),
    (0, 0, 90),
    (0, 0, 110),
    (0, 80, 100),
    (0, 0, 230),
    (119, 11, 32),
    (0, 0, 142)
]

#trainid

palette = [
    (128,64,128),
    (244,35,232),
    (70,70,70),
    (102,102,156),
    (190,153,153),
    (153,153,153),
    (250,170, 30),
    (220,220,  0),
    (107,142, 35),
    (152,251,152),
    ( 70,130,180),
    (220, 20, 60),
    (255,  0,  0),
    (  0,  0,142),
    (  0,  0, 70),
    (  0, 60,100),
    (  0, 80,100),
    (  0,  0,230),
    (119, 11, 32),
]


print(len(palette))


19


In [9]:
#data_root = 'results_check/configs/' + config + '/'
#anno_roots = ['frankfurt/', 'munster/', 'lindau/']

data_root = './results_output/segformer/frankfurt/'
anno_roots = ['']

print(palette)
print(classes)
for anno_root in anno_roots:
    save_root = data_root + anno_root
    rgbToAnnotation(data_root, anno_root, palette)
    palette_colour_to_gray_save(data_root, anno_root, save_root)
    print(save_root, "Complete")

[(128, 64, 128), (244, 35, 232), (70, 70, 70), (102, 102, 156), (190, 153, 153), (153, 153, 153), (250, 170, 30), (220, 220, 0), (107, 142, 35), (152, 251, 152), (70, 130, 180), (220, 20, 60), (255, 0, 0), (0, 0, 142), (0, 0, 70), (0, 60, 100), (0, 80, 100), (0, 0, 230), (119, 11, 32)]
('road', 'sidewalk', 'building', 'wall', 'fence', 'pole', 'traffic light', 'traffic sign', 'vegetation', 'terrain', 'sky', 'person', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle', 'bicycle')
./results_output/segformer/frankfurt/
./results_output/segformer/frankfurt/ Complete


In [11]:
import os
import cv2
import numpy as np
from collections import Counter

# Function to calculate IoU
def calculate_iou_class(pred_mask, gt_mask, class_id):
    pred_class = (pred_mask == class_id)
    gt_class = (gt_mask == class_id)
    intersection = np.logical_and(pred_class, gt_class)
    union = np.logical_or(pred_class, gt_class)
    #print((np.sum(intersection)), np.sum(union))
    iou = np.sum(intersection) / np.sum(union)
    return iou

def calculate_acc(pred_mask, gt_mask):
    acc = np.sum(pred_mask == gt_mask)
    total = pred_mask.size
    return acc / total

In [ ]:
import os
# Path to the folder containing prediction masks and ground truth masks
prediction_folder = './results_output/model_test_denoising'
ground_truth = '../../../../../../mnt/data/cityscapes/gtFine/val/frankfurt/frankfurt_000000_000294_gtFine_labelIds.png'
model_prediction = './results_output/frankfurt_000000_000294_leftImg8bit.png'

#ground_truth = './results_check/configs/segformer/frankfurt/frankfurt_000000_000294_leftImg8bit.png'

# List all image files in the folders
prediction_files = os.listdir(prediction_folder)

# Find corresponding ground truth mask
image_mask = cv2.imread(ground_truth, cv2.IMREAD_GRAYSCALE)

print(Counter(image_mask.flatten()).keys())
classes = set(Counter(image_mask.flatten()).keys())  # Find all classes to test
ignore = {0,1,2,3,4,5,6,9,10,14,15,16,18,29,30,-1}
new_classes = list(classes - ignore)
print(new_classes)


#Prompt test
x = "RAW photo, European street, dashcam footage with trees and cars (road:1.2), 8k uhd, dslr, soft lighting, dashcam quality, Jujifilm XT3".split(', ')
y = ['Denoising ' + str(i) for i in [0.4,0.5,0.6]]

#Denoising Test (Also used for model tests)
x = ['CFG 5.0', 'CFG 7.0', 'CFG 10.0', 'CFG 12.0', 'CFG 15.0']
#y = [0.3,0.4,0.5]

#Seed Test
#x = [f"Seed {i+1}" for i in range(10)]
#y = ["CFG 5", "CFG 10"]

def calculate_result(pred_mask, gt_mask, labels, file_desc):
    iou_values = np.empty(len(labels))  
    if pred_mask.shape != gt_mask.shape:
        print(f"Error: Image sizes do not match")
        return

    # Calculate IoU for each class
    for ii, class_id in enumerate(labels):
        iou = calculate_iou_class(pred_mask, gt_mask, class_id)
        iou_values[ii] = iou
        acc = calculate_acc(pred_mask, gt_mask)
        #print(f"IoU for {class_id}: {iou:.4f}")
    return (np.mean(iou_values), file_desc, acc)

results = []

model_mask = cv2.imread(model_prediction, cv2.IMREAD_GRAYSCALE)
results.append(calculate_result(model_mask, image_mask, new_classes, "Model prediction"))

for i, pred_file in enumerate(prediction_files):
    pred_mask = cv2.imread(os.path.join(prediction_folder, pred_file), cv2.IMREAD_GRAYSCALE)
    results.append(calculate_result(pred_mask, image_mask, new_classes, f'{x[i % len(x)]} {y[i // len(x)]}'))

results.sort()
for i in results:
    print(f"IoU: {i[0]:.4f} Acc: {i[2]:.4f} File: {i[1]}")

In [ ]:
ignore = {0,1,2,3,4,5,6,9,10,14,15,16,18,29,30,-1}

#Comparing Frankfurt results
ground_truth = '../../../../../../mnt/data/cityscapes/gtFine/val/frankfurt/'
original_pred = './results_check/configs/segformer/frankfurt/'
prediction_files = os.listdir(original_pred)

mod_real_seg_pred = './results_output/frankfurt_real_seg/'
mod_comp_seg_pred = './results_output/frankfurt_comp_seg/'
results = np.empty([3,267])

for i, filename in enumerate(prediction_files):
    # Read in original prediction
    orignal_pred_image = cv2.imread(os.path.join(original_pred, filename), cv2.IMREAD_GRAYSCALE)

    #Read in modified predictions
    mod_real_pred_image = cv2.imread(os.path.join(mod_real_seg_pred, filename), cv2.IMREAD_GRAYSCALE)
    mod_comp_pred_image = cv2.imread(os.path.join(mod_comp_seg_pred, filename), cv2.IMREAD_GRAYSCALE)

    #Read in ground truth
    ground_truth_image = cv2.imread(os.path.join(ground_truth, filename.replace("leftImg8bit", 'gtFine_labelIds')), cv2.IMREAD_GRAYSCALE)
    gt_labels = list(set(np.unique(ground_truth_image)) - ignore)

    results[0][i] = calculate_result(orignal_pred_image,ground_truth_image, gt_labels, "Original Image")[0]
    results[1][i] = calculate_result(mod_real_pred_image,ground_truth_image, gt_labels, "Synthetic image (real segmentation)")[0]
    results[2][i] = calculate_result(mod_comp_pred_image,ground_truth_image, gt_labels, "Synthetic image (predicted segmentation)")[0]

print(np.mean(results[0]))
print(np.mean(results[1]))
print(np.mean(results[2]))